In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_path="/content/drive/My Drive/Machine_Translation"

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
# read phrases from english_telugu_data.txt file
english_sentences = []
telugu_sentences = []
import pandas as pd
df=pd.read_csv("en-te_parallel_corpus.csv")
english_sentences=df["en_sent"].iloc[0:500]
telugu_sentences=df["te_sent"].iloc[0:500]


In [ ]:
data = pd.DataFrame({"english_sentences":english_sentences,"telugu_sentences":telugu_sentences})
data

,english_sentences,telugu_sentences
0,Three members of a family were killed in the i...,ఓకే కుటుంబానికి చెందిన ముగ్గురు మృతిచెందటంతో ఈ...
1,"In Monaco-Ville, street signs are printed in b...","మొనాకో-విల్లెలో, ఫ్రెంచ్, మోనెగస్క్యూ రెండింటి..."
2,The same has been confirmed in a report carrie...,ప్రముఖ బీబీసీ నివేదిక ఇదే విషయం పేర్కొంది.
3,In this state they can easily survive one year...,సంవత్సరానికి అదనంగా ఒక రోజునో లేక ఒక నెలనో చేర...
4,Annual International Yoga Day is a modern cele...,ఇలా అంతర్జాతీయ యోగ దినోత్సవం భారతీయ సంస్కార ప్...
...,...,...
495,Each State delegation in the Congress casts on...,ప్రతి రాష్ట్రం కాంగ్రెస్లో ఓటు ఉంటుంది.
496,The NTR biopic is being made into a two-part f...,ఎన్టీఆర్ బయోపిక్ రెండు భాగాలుగా రూపొందుతున్న స...
497,The court had also served notice on the CBI im...,ఈ కేసులో సిబిఐకి కూడా అప్పుడు కోర్టు నోటీసులు ...
498,North Korea has carried out repeated nuclear a...,"ఉత్తర కొరియా ఇటీవల పదేపదే మిసైల్, అణుపరీక్షలు ..."


In [ ]:
# clean english sentances
def clean_eng(text):
    # Lowercase all characters
    text = text.lower()
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove extra spaces
    text= text.strip()

    return text

In [ ]:
# clean telugu sentances
def clean_tel(text):
    # Lowercase all characters
    text = text.lower()
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove Telugu numbers from text
    text = re.sub("[౦౧౨౩౪౫౬౭౮౯]", '', text)
    # Remove extra spaces
    text= text.strip()
    text = 'START_ '+ text + ' _END'
    return text

In [ ]:
# clean text
data_df = data.copy()
data_df["english_sentences"] = data_df["english_sentences"] .apply(lambda x: clean_eng(x))
data_df["telugu_sentences"] = data_df["telugu_sentences"] .apply(lambda x: clean_tel(x))

In [ ]:
# Vocabulary of English
all_eng_words=set()
for eng in data_df.english_sentences:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
# Vocabulary of Telugu 
all_telugu_words=set()
for tel in data_df.telugu_sentences:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in data_df.english_sentences:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

20

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in data_df.telugu_sentences:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

92

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens

(2306, 3288)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

3289

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
data_df = shuffle(data_df)
data_df.head(10)

,english_sentences,telugu_sentences
370,such candidates can fill the form from octobe...,START_ అర్హత గల అభ్యర్థులు నవంబర్ నుంచి నవంబర...
237,five persons have been arrested in the matter ...,START_ ఐదుగురిని ఈ కేసులో అరెస్టు చేసినట్లు ఆయ...
273,an exchange discount of up to is also available,START_ వరకు ఎక్స్చేంజ్ డిస్కౌంట్ _END
386,in puntland higher education is provided by th...,START_ పుంట్లాండులో ఉన్నత విద్యను పుంట్లాండు స...
254,the season will feature matches in a week period,START_ సుమారు వారాల పాటు మ్యాచ్‌లు జరగనున్నా...
225,pdp won two seats whereas bjp and congress bag...,START_ అలాగే రెండుసీట్లను కాంగ్రెస్ ఒక్కో సీటు...
153,nirbhay gives indian armed forces deepstrike c...,START_ నిర్భయ్ క్షిపణికి గాల్లోనూ భూమ్మీద నుంచ...
421,director anil ravipudis upcoming film sarileru...,START_ సూపర్ స్టార్ మహేశ్ బాబు అనిల్ రావిపూడి...
267,the nominations were announced by actors tiffa...,START_ నామినేష‌న్లను వెల్ల‌డించిన ఆండీ సెర్కిస...
2,the same has been confirmed in a report carrie...,START_ ప్రముఖ బీబీసీ నివేదిక ఇదే విషయం పేర్కొం...


In [ ]:
# Train - Test Split
X, y = data_df.english_sentences, data_df.telugu_sentences
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((450,), (50,))

In [ ]:
X_train.to_pickle(project_path+'X_train.pkl')
X_test.to_pickle(project_path+'X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

#Encoder - Decoder Model Architecture

In [ ]:
latent_dim = 50

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
batch_size = 128
epochs = 30
train_samples_steps = len(X_train) // batch_size
val_samples_steps = len(X_test) // batch_size

In [ ]:
# generate train and test datra
train_gen = generate_batch(X_train, y_train, batch_size = batch_size)
test_gen = generate_batch(X_test, y_test, batch_size = batch_size)

In [ ]:
# Defining a helper function to save the model after each epoch 
# in which the loss decreases 
filepath = project_path+'NMT_model_enc_dec.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# Defining a helper function to reduce the learning rate each time 
# the learning plateaus 
reduce_alpha = ReduceLROnPlateau(monitor ='val_loss', factor = 0.2,patience = 1, min_lr = 0.001)
# stop traning if there increase in loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
callbacks = [checkpoint, es, reduce_alpha] 

In [ ]:
# train the model
model.fit_generator(generator = train_gen,steps_per_epoch = train_samples_steps,epochs=epochs,validation_data = test_gen,validation_steps = val_samples_steps,callbacks = callbacks)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


2/3 [===================>..........] - ETA: 0s - loss: 0.5740 - acc: 0.0209

InvalidArgumentError: ignored

# Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Decode Sample Sequences

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation on Train Dataset

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
for k in range(10):
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_test[k:k+1].values[0])
    print('Actual Telugu Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Telugu Translation:', decoded_sentence[:-4])
    print("\n")

Input English sentence: madhu chopra also revealed that priyanka had showed her the dress before wearing it to the grammys
Actual Telugu Translation:  ప్రియాంక వేసుకోవడానికి ముందుకు కూడా తనకు శాంపిల్ చూపించనట్టుగా మధు చోప్రా వెల్లడించింది 
Predicted Telugu Translation:  పడ్డారు తెలుగుతమిళమలయాళం సిరీస్‌లో సిరీస్‌లో నార


Input English sentence: sanand was also a member of national river ganga basin authority during the upa tenure
Actual Telugu Translation:  యూపీఏ హయాంలో ఏర్పాటైన నేషనల్ రివర్ గంగా బేసిన్ అథారిటీలో స్వామి సనంద్ సభ్యునిగా కూడా కొనసాగారు 
Predicted Telugu Translation:  అన్నాడు సమానం సమానం ప్రశాంత్‌ ఐటీఐ రక్షణ మెహ‌రీన్ మెహ‌


Input English sentence: this helps to make the christian congregation attractive to others who wish to be free of conflict
Actual Telugu Translation:  దాన్ని చూసి ఇతరులు కూడా క్రైస్తవ సంఘానికి రావడానికి ఇష్టపడతారు 
Predicted Telugu Translation:  నినాదం టైగర్ టైగర్ మోడల్ తెస్తున్నాం స్టోరేజీ నటిస్తున్


Input English sentence: he said that the movie will